### Introduction
This project attempts to analyse the possibility of default on credit card payments in Taiwan. The dataset used was from the UCI archives. 
Link - https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients


#### Imports

In [ ]:
import pandas as pd;
import numpy as np;
import seaborn as sns;
import sklearn as sk;
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
from sklearn.feature_selection import SelectKBest,chi2,f_classif
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

### Read Inputs and Examine

In [ ]:
# Set console width to max
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 100)

credit = pd.read_csv(r"C:\Users\Dell\Desktop\Assignments\Adv Stats\Project\data.csv")
print(credit.describe().T)

In [ ]:
# Any null values?
print(credit.isnull().sum())

We can make a few observations based on this data. 
The target variable is a categorical variable, of defaulters, with 0 indicating no default and 1 indicating default. There is no missing data and about 22.1%(0.221200) of the entries are defaulters. So, our model should atleast have accuracy than 77.98% since the data already is skewed. 

In [ ]:
print(pd.Series({c: credit[c].unique() for c in credit}))

As we continue our analysis, we see that some fields have not been defined in the dataset. For instance, we do not know what EDUCATION fields 5,6 and 0 mean. We could consider adding it to the Others field, i.e. 4. Similarly with MARRIAGE, we do not know what 3 indicates. PAY_0 to PAY_6 also have extra fields of 0 and -2, which are undefined in the dataset.

We could choose to ignore this issue, but if we do, and these unknown fields may be picked up in the train dataset. If their size is substantial, we will be unable to interpret the result. If not, it will contribute to model inaccuracy. If it gets picked up in the test dataset too, it may contribute to overfitting. 

In [ ]:
print(credit.EDUCATION.value_counts())
print(credit.MARRIAGE.value_counts())
print(credit.PAY_0.value_counts())

The unknown fields in EDUCATION, MARRIAGE are small in number, and we can safely delete it, since it doesn't make sense and there is already an 'Other' field defined for each of them. However, with PAY_0, we see a large chunk of the data of type 0 and -2. This is seen in all the 'Pay_x' fields. 

I shall make a logical assumption here - if 1 refers to Payment delay for 1 month, 2 refers to Payment delay for 2 months, then most likely, 0  refers to Payment on time, -1 refers to Payment made 1 month in advance, -2 refers to Payment made 2 months in advance, with the last two being a debit style payment. To make things simpler, I shall combine 0, -1 and -2 as payment on time or 0, since this study is to do with the probability of default and not early payment of some sort.


In [ ]:
credit = credit[(credit.EDUCATION != 5) & (credit.EDUCATION != 6) & (credit.EDUCATION != 0)]
credit = credit[(credit.MARRIAGE != 3)]

print("Before Replacement:\n",credit.PAY_0.value_counts())
credit.PAY_0 = credit['PAY_0'].replace([-1,-2],0)
print("After Replcement:\n",credit.PAY_0.value_counts())

Since this working correctly, I shall do this for all the Pay_x fields

In [ ]:
credit.PAY_2 = credit['PAY_2'].replace([-1,-2],0)
credit.PAY_3 = credit['PAY_3'].replace([-1,-2],0)
credit.PAY_4 = credit['PAY_4'].replace([-1,-2],0)
credit.PAY_5 = credit['PAY_5'].replace([-1,-2],0)
credit.PAY_6 = credit['PAY_6'].replace([-1,-2],0)

Interestingly, PAY_1 is missing and we have PAY_0

In [ ]:
corr_df = credit.corr()
print(corr_df)

In [ ]:

var = ['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']

plt.figure(figsize = (8,8))
plt.title('Amount of bill statement (Apr-Sept) \ncorrelation plot (Pearson)')
corr = credit[var].corr()
sns.heatmap(corr,xticklabels=corr.columns,yticklabels=corr.columns,linewidths=.1,vmin=-1, vmax=1)
plt.show()

var = ['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5']

plt.figure(figsize = (8,8))
plt.title('Amount of previous payment (Apr-Sept) \ncorrelation plot (Pearson)')
corr = credit[var].corr()
sns.heatmap(corr,xticklabels=corr.columns,yticklabels=corr.columns,linewidths=.1,vmin=-1, vmax=1)
plt.show()

var = ['PAY_0','PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
plt.figure(figsize = (8,8))
plt.title('Repayment status (Apr-Sept) \ncorrelation plot (Pearson)')
corr = credit[var].corr()
sns.heatmap(corr,xticklabels=corr.columns,yticklabels=corr.columns,linewidths=.1,vmin=-1, vmax=1)
plt.show()


We can see that PAY_x and BILL_AMTx have high correlation and can be dropped. I will only retain PAY_0, PAY_6 (correlation is less than 0.5) and BILL_AMT1. Since we have already adjusted for correlation, I shall not perform PCA. 
Next, we shall create dummy variables for the categorical fields and split the dataset. Since there is no instruction, I shall split it 2/3rd:1/3rd and perform MinMax Scaling. I will not scale the categorical y since it is senseless. This also means that I do not need to inverse transform the scaling.

In [ ]:
categs=['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
for c in categs:
    credit[c]=credit[c].astype('category')
credit_wd=pd.get_dummies(credit, drop_first=True)

y=credit_wd['default payment next month']
X_=credit_wd.drop(columns=['default payment next month'])

X = X_.loc[:,['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'SEX_2', 'EDUCATION_2', 'EDUCATION_3', 'EDUCATION_4', 'MARRIAGE_1', 'MARRIAGE_2', 'PAY_0_1', 'PAY_0_2', 'PAY_0_3', 'PAY_0_4', 'PAY_0_5', 'PAY_0_6', 'PAY_0_7', 'PAY_0_8', 'PAY_6_2', 'PAY_6_3', 'PAY_6_4', 'PAY_6_5', 'PAY_6_6', 'PAY_6_7', 'PAY_6_8']
]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1, test_size=0.33)
minMaxScaler = MinMaxScaler(feature_range=(0, 1));
X_train = minMaxScaler.fit_transform(X_train);
X_test = minMaxScaler.transform(X_test);


### Algorithms
Since this is a classification problem, we can use Logistical Regression, Random Forests, Decision Trees, k Nearest Neighbours and Neural Network to classify. In order to decide which is optimal, I shall run all of them and see which performs better than the others. First, Logistic Regression

In [ ]:
mod_lin = LogisticRegression(solver='lbfgs',random_state=1,max_iter=250)
mod_lin.fit(X_train, y_train)

print("Log Reg - Training metrics")
print('Accuracy:', accuracy_score(y_train, mod_lin.predict(X_train)))
print('Precision:', precision_score(y_train, mod_lin.predict(X_train)))
print('Recall:', recall_score(y_train, mod_lin.predict(X_train)))
print("Conf Matrix:\n", sk.metrics.confusion_matrix(pd.DataFrame(y_train),pd.DataFrame(mod_lin.predict(X_train))))

print("Log Reg - Testing metrics")
print('Accuracy:', accuracy_score(y_test, mod_lin.predict(X_test)))
print('Precision:', precision_score(y_test, mod_lin.predict(X_test)))
print('Recall:', recall_score(y_test, mod_lin.predict(X_test)))
print("Conf Matrix:\n", sk.metrics.confusion_matrix(pd.DataFrame(y_test),pd.DataFrame(mod_lin.predict(X_test))))
print('AUC:',roc_auc_score(y_test, mod_lin.predict(X_test)))
print('F1 Score:',f1_score(y_test, mod_lin.predict(X_test)))

Straight away, we see that the Logistic Regression performs well, with high testing and training accuracy. There is no overfitting. We see a good AUC of 0.655 and F1 score of 0.47. We shall check if RandomForest beats this model.

In [ ]:
rf=RandomForestClassifier(random_state=1,criterion='entropy',n_estimators=50,verbose=False)
rf.fit(X_train, y_train)

print('Random Forest - Training set metrics:')
print('Accuracy:', accuracy_score(y_train, rf.predict(X_train)))
print('Precision:', precision_score(y_train, rf.predict(X_train)))
print('Recall:', recall_score(y_train, rf.predict(X_train)))
print("Conf Matrix:\n", confusion_matrix(pd.DataFrame(y_train),pd.DataFrame(rf.predict(X_train))))

print('Random Forest - Test set metrics:')
print('Accuracy:', accuracy_score(y_test, rf.predict(X_test)))
print('Precision:', precision_score(y_test, rf.predict(X_test)))
print('Recall:', recall_score(y_test, rf.predict(X_test)))
print("Conf Matrix:\n", confusion_matrix(pd.DataFrame(y_test),pd.DataFrame(rf.predict(X_test))))

There is overfitting of the data, so I shall try some combinations to overcome it

In [ ]:
rf=RandomForestClassifier(random_state=1,n_estimators=50,criterion='entropy',max_depth=10,verbose=False)
rf.fit(X_train, y_train)

print('Random Forest - Training set metrics:')
print('Accuracy:', accuracy_score(y_train, rf.predict(X_train)))
print('Precision:', precision_score(y_train, rf.predict(X_train)))
print('Recall:', recall_score(y_train, rf.predict(X_train)))
print("Conf Matrix:\n", confusion_matrix(pd.DataFrame(y_train),pd.DataFrame(rf.predict(X_train))))

print('Random Forest - Test set metrics:')
print('Accuracy:', accuracy_score(y_test, rf.predict(X_test)))
print('Precision:', precision_score(y_test, rf.predict(X_test)))
print('Recall:', recall_score(y_test, rf.predict(X_test)))
print("Conf Matrix:\n", confusion_matrix(pd.DataFrame(y_test),pd.DataFrame(rf.predict(X_test))))

In [ ]:
rf=RandomForestClassifier(random_state=1,n_estimators=200,criterion='entropy',max_depth=5,verbose=False)
rf.fit(X_train, y_train)

print('Random Forest - Training set metrics:')
print('Accuracy:', accuracy_score(y_train, rf.predict(X_train)))
print('Precision:', precision_score(y_train, rf.predict(X_train)))
print('Recall:', recall_score(y_train, rf.predict(X_train)))
print("Conf Matrix:\n", confusion_matrix(pd.DataFrame(y_train),pd.DataFrame(rf.predict(X_train))))

print('Random Forest - Test set metrics:')
print('Accuracy:', accuracy_score(y_test, rf.predict(X_test)))
print('Precision:', precision_score(y_test, rf.predict(X_test)))
print('Recall:', recall_score(y_test, rf.predict(X_test)))
print("Conf Matrix:\n", confusion_matrix(pd.DataFrame(y_test),pd.DataFrame(rf.predict(X_test))))
print("AUC:", roc_auc_score(y_test, rf.predict(X_test)))
print('F1 Score:',f1_score(y_test, rf.predict(X_test)))

After trying different combinations of parameters, this is the best combination that I could find, with barely no overfitting. This model performs similarly to the Logistical Regression in terms of Accuracy, however, with a very low recall score, it has a lower AUC and F1 score than Logistical Regression. Next, we shall check k-Nearest Neighbor.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=8, leaf_size=30,metric='minkowski', algorithm= 'auto')
knn.fit(X_train, y_train)

print('kNN - Training set metrics:')
print('Accuracy:', accuracy_score(y_train, knn.predict(X_train)))
print('Precision:', precision_score(y_train, knn.predict(X_train)))
print('Recall:', recall_score(y_train, knn.predict(X_train)))
print("Conf Matrix:\n", confusion_matrix(pd.DataFrame(y_train),pd.DataFrame(knn.predict(X_train))))

print('kNN - Test set metrics:')
print('Accuracy:', accuracy_score(y_test, knn.predict(X_test)))
print('Precision:', precision_score(y_test, knn.predict(X_test)))
print('Recall:', recall_score(y_test, knn.predict(X_test)))
print("Conf Matrix:\n", confusion_matrix(pd.DataFrame(y_test),pd.DataFrame(knn.predict(X_test))))
print("AUC:", roc_auc_score(y_test, knn.predict(X_test)))
print('F1 Score:',f1_score(y_test, knn.predict(X_test)))

With a few trials, these are the optimum parameters for kNN model for this dataset. This model has similar accuracy as the Random Forest, but better Precision Recall scores. But, they are marginally lower than the Logistic Regression AUC and F1 values. Additionally, it must be noted that this method was the most computationally intensive method sofar. Next, I shall try Decision Trees

In [ ]:
dt = DecisionTreeClassifier(max_depth=7, random_state=1)
dt.fit(X_train, y_train)

print('Decision Tree - Training set metrics:')
print('Accuracy:', accuracy_score(y_train, dt.predict(X_train)))
print('Precision:', precision_score(y_train, dt.predict(X_train)))
print('Recall:', recall_score(y_train, dt.predict(X_train)))
print("Conf Matrix:\n", confusion_matrix(pd.DataFrame(y_train),pd.DataFrame(dt.predict(X_train))))

print('Decision Tree - Test set metrics:')
print('Accuracy:', accuracy_score(y_test, dt.predict(X_test)))
print('Precision:', precision_score(y_test, dt.predict(X_test)))
print('Recall:', recall_score(y_test, dt.predict(X_test)))
print("Conf Matrix:\n", confusion_matrix(pd.DataFrame(y_test),pd.DataFrame(dt.predict(X_test))))
print("AUC:", roc_auc_score(y_test, dt.predict(X_test)))
print('F1 Score:',f1_score(y_test, dt.predict(X_test)))


This method has done better than kNN, but is still short of the F1 scores and AUC values of Logistic Regression. It is interesting to see that the simplest algorithm has the best performance yet. Lastly, Neural Network

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(400,200,100), activation='relu', solver='lbfgs', random_state=1, verbose=False)
mlp.fit(X_train, y_train)


print('Neural Network - Training set metrics:')
print('Accuracy:', accuracy_score(y_train, mlp.predict(X_train)))
print('Precision:', precision_score(y_train, mlp.predict(X_train)))
print('Recall:', recall_score(y_train, mlp.predict(X_train)))
print("Conf Matrix:\n", confusion_matrix(pd.DataFrame(y_train),pd.DataFrame(mlp.predict(X_train))))

print('Neural Network - Test set metrics:')
print('Accuracy:', accuracy_score(y_test, mlp.predict(X_test)))
print('Precision:', precision_score(y_test, mlp.predict(X_test)))
print('Recall:', recall_score(y_test, mlp.predict(X_test)))
print("Conf Matrix:\n", confusion_matrix(pd.DataFrame(y_test),pd.DataFrame(mlp.predict(X_test))))
print("AUC:", roc_auc_score(y_test, mlp.predict(X_test)))
print('F1 Score:',f1_score(y_test, mlp.predict(X_test)))


Finally, we see that the Neural Network has outperformed the Logistical Regression. Although Recall scores are still on the low side, it has a high accuracy, a moderate precision, and the highest AUC and F1 scores so far. There is also no overfitting. 

To understand the contribution of each variable in the regression, we need to check correlations between variables and target, I have used Chi Squared Test for the categorical variables and Anova f test for continuous variables. I am using SelectKBest, which gives me the best k features by using Chi Squared and F tests. If the value of the tests are high, it means that the feature and target variable are highly correlated & their relative variance is high.
I have also set k to 'all', to give me the complete list of features with their performance. 


In [ ]:
print(list(X.columns))
X_cont = X.loc[:,['LIMIT_BAL','BILL_AMT1','PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']] #Continuous variables
X_cate = X.iloc[:,9:] #Categorical Variables
fs_cont = SelectKBest(score_func=f_classif, k='all')
fs_cont.fit(X_cont,y)
print('F test with pvalues')
for i in range(len(fs_cont.scores_)):
    print('%s: %f - %f' % (X_cont.columns[i], fs_cont.scores_[i], fs_cont.pvalues_[i]))

fs_cat = SelectKBest(score_func=chi2, k='all')
fs_cat.fit(X_cate,y)
print('Chi 2 test with pvalues')
for i in range(len(fs_cat.scores_)):
    print('%s: %f - %f' % (X_cate.columns[i], fs_cat.scores_[i], fs_cat.pvalues_[i]))


We immediately see that LIMIT_BAL has a massive F test value, indicating that the output variable is largely controlled by this feature. Similarly, we see that PAY_0_2, PAY_6_2 and PAY_0_3 have a significant effect as well. As a quick test, I shall run a Logistic Regression with only these parameters and observe their results. 

In [ ]:
X_ = X_.loc[:,['LIMIT_BAL','PAY_0_2','PAY_6_2','PAY_0_3']]
X_train, X_test, y_train, y_test = train_test_split(X_, y, stratify=y, random_state=1, test_size=0.33)

minMaxScaler = MinMaxScaler(feature_range=(0, 1));
X_train = minMaxScaler.fit_transform(X_train);
X_test = minMaxScaler.transform(X_test);

mod_lin = LogisticRegression(solver='lbfgs',random_state=1,max_iter=250)
mod_lin.fit(X_train, y_train)

print("Log Reg - Training metrics")
print('Accuracy:', accuracy_score(y_train, mod_lin.predict(X_train)))
print('Precision:', precision_score(y_train, mod_lin.predict(X_train)))
print('Recall:', recall_score(y_train, mod_lin.predict(X_train)))
print("Conf Matrix:\n", sk.metrics.confusion_matrix(pd.DataFrame(y_train),pd.DataFrame(mod_lin.predict(X_train))))

print("Log Reg - Testing metrics")
print('Accuracy:', accuracy_score(y_test, mod_lin.predict(X_test)))
print('Precision:', precision_score(y_test, mod_lin.predict(X_test)))
print('Recall:', recall_score(y_test, mod_lin.predict(X_test)))
print("Conf Matrix:\n", sk.metrics.confusion_matrix(pd.DataFrame(y_test),pd.DataFrame(mod_lin.predict(X_test))))
print('AUC:',roc_auc_score(y_test, mod_lin.predict(X_test)))
print('F1 Score:',f1_score(y_test, mod_lin.predict(X_test)))

We see that the AUC and F1 Scores are very similar to the actual Logistical Regression done earlier. Even Accuracy is comparable. Hence, we can say that these variables explain the possibility of default, the most. 

### Conclusion
To summarize, we analysed the data for undefined values and adjusted for them, and checked for missing data. Then, we examined correlation between features and built heatmaps for groups of data that were suspect. After eliminating these features and scaling the rest, we ran Logistical, Random Forest, k-NN, Decision Trees and Neural Network, to check which model fit the classification the best. The models were tested with testing dataset and adjusted for overfitting.

With the results of the analysis, we see that the Neural Network performs the best for this dataset, with an accuracy of 82.1% and F1 score of 0.476. AUC score is also the highest at 65.85%. But the far simpler Logistic Regression may also be examined for a lightweight model with similar results. 
Additionally, we also see that the features - 'LIMIT_BAL','PAY_0_2','PAY_6_2','PAY_0_3' have the most correlation with the Default Variable and an extension of this project could be to re-examine each algorithm with these features alone.